# DOTA 2

Until patch 7.07 there were 115 heroes, which it viewed from their primary attributes, they were divided into 3 groups like Strength (STR), Agility (AGI) and Intelligent (INT).

Each hero has different stats such as Base STR, Base AGI, Base INT, STR Growth, Min DMG, Max DMG, Movement Speed, etc. The data that used in this time comes from [https://dota2.gamepedia.com/Table_of_hero_attributes](https://dota2.gamepedia.com/Table_of_hero_attributes).

For example was Centaur, he was a hero with STR type, and the value of Base STR and STR Growth stats was very high but there was alot of heroes who have high stats on one of the attributes but are not the hero type.

Like Ogre Magi, he is an INT type, but his Base STR, STR Growth and Max STR values are higher than his INT stats. Now, all heroes based on available stats need to be classify.

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow.python.util import deprecation
deprecation._PRINT_DEPRECATION_WARNINGS = False

from tensorflow.python.keras.models import Model, Sequential
from tensorflow.python.keras.layers import Input, Activation, Dense
from tensorflow.python.keras.optimizers import SGD
from tensorflow.python.keras.utils.np_utils import to_categorical

First parse the web page and convert the HTML table to CSV. There are 115 pieces of data. Later there will be **16 heroes that will be taken as validation** and **the rest will be used as training**.

For data validation, take some heroes who have strange stats like Jakiro, Winter Wyvern (high STR stats but the actually was INT hero), Phoenix, IO (low STR but the actually was STR hero), Bloodseeker, Undying, etc.

Stats that (in my opinion) were irrelevant will be throw away, such as Day Vision, Night Vision, Collision Size, Legs. So, there are 22 features that will be used to do the classification. The hero type will be use as a target which has a value such as 0 for STR, 1 for AGI and 2 for INT.

## Features Correlation

From the 22 features above, it may be that not all features have contributed to determine the type of hero or can be said it was up and down of a feature did not affect the type of hero.

To find these features a ***correlation matrix*** method can be used for each variable.

***What is correlation matrix? Correlation matrix contains values between -1 and +1. For example if the correlation between variables X and Y is -1, then when the value of X goes down, the value of Y will rise.***

***If it is +1, then the relationship between X and Y is linear. But if the correlation value gets closer to 0, then the variable can be said not to affect one another.***

In [2]:
# Pandas read CSV
sf_train = pd.read_csv('data/5_training_data.csv')

In [3]:
# Correlation matrix for target
corr_matrix = sf_train.corr()

# Show how corelation matrix table
print(corr_matrix)

                     type   baseStr  strGrowth    maxStr   baseAgi  agiGrowth  \
type             1.000000 -0.542666  -0.542322 -0.581514 -0.038086  -0.123072   
baseStr         -0.542666  1.000000   0.590398  0.719891 -0.332447  -0.287354   
strGrowth       -0.542322  0.590398   1.000000  0.985228 -0.393197  -0.195093   
maxStr          -0.581514  0.719891   0.985228  1.000000 -0.408670  -0.228741   
baseAgi         -0.038086 -0.332447  -0.393197 -0.408670  1.000000   0.694799   
agiGrowth       -0.123072 -0.287354  -0.195093 -0.228741  0.694799   1.000000   
maxAgi          -0.110488 -0.313907  -0.251592 -0.282962  0.804062   0.986261   
baseInt          0.688410 -0.325844  -0.385914 -0.401006 -0.262319  -0.339530   
intGrowth        0.657814 -0.347587  -0.447942 -0.458961 -0.315120  -0.429123   
maxInt           0.699666 -0.361478  -0.458466 -0.470959 -0.320535  -0.432660   
totalBaseAttr    0.179483  0.113566  -0.311589 -0.243860  0.539259   0.189111   
totalAttrGrowth  0.063110 -0

In [4]:
# Show corelation matrix affect toward type
print(corr_matrix['type'])

type               1.000000
baseStr           -0.542666
strGrowth         -0.542322
maxStr            -0.581514
baseAgi           -0.038086
agiGrowth         -0.123072
maxAgi            -0.110488
baseInt            0.688410
intGrowth          0.657814
maxInt             0.699666
totalBaseAttr      0.179483
totalAttrGrowth    0.063110
totalMaxAttr       0.114713
moveSpeed          0.067136
baseArmor         -0.113657
minDmg            -0.384719
maxDmg            -0.312799
range              0.687704
baseAttackTime    -0.297982
attackPoint       -0.106053
attackBackswing    0.011558
turnRate          -0.034759
regeneration      -0.090677
Name: type, dtype: float64


This time all features that have a correlation value smaller than 0.1 or -0.1 such as regeneration rate, turn rate, movement speed and strangely here the base AGI, AGI growth and max AGI have very small values will be discard.

In [5]:
# Drop unnecessary columns
sf_train.drop(sf_train.columns[[5, 12, 14, 21, 22, 23]], axis=1, inplace=True)

# Show current columns after drop the unnecessary
print(sf_train.head())

                 name  type  baseStr  strGrowth  maxStr  agiGrowth  maxAgi  \
0             Abaddon     0       23        2.6    85.4        1.5    53.0   
1  Ancient Apparition     2       18        1.7    58.8        2.2    72.8   
2           Anti-Mage     1       22        1.5    58.0        2.8    89.2   
3                 Axe     0       25        2.8    92.2        2.2    72.8   
4                Bane     2       23        2.4    80.6        2.4    80.6   

   baseInt  intGrowth  maxInt  totalBaseAttr  totalMaxAttr  baseArmor  minDmg  \
0       21        2.0    69.0             61         207.4       1.43      55   
1       25        2.6    87.4             63         219.0       1.86      44   
2       15        1.8    58.2             59         205.4       2.14      49   
3       18        1.6    56.4             63         221.4       1.86      49   
4       23        2.4    80.6             69         241.8       4.29      60   

   maxDmg  range  baseAttackTime  attackPoin

In [6]:
# Pandas read Validation CSV
sf_val = pd.read_csv('data/5_validation_data.csv')

# Drop unnecessary columns
sf_val.drop(sf_val.columns[[5, 12, 14, 21, 22, 23]], axis=1, inplace=True)

# Show current columns after drop the unnecessary
print(sf_val.head())

           name  type  baseStr  strGrowth  maxStr  agiGrowth  maxAgi  baseInt  \
0     ArcWarden     1       24        3.0    96.0        1.8    58.2       24   
1        Jakiro     2       25        2.6    87.4        1.2    38.8       28   
2  WinterWyvern     2       24        2.4    81.6        1.9    61.6       25   
3   Bloodseeker     1       23        2.7    87.8        3.0    96.0       18   
4       Phoenix     0       19        3.2    95.8        1.3    43.2       18   

   intGrowth  maxInt  totalBaseAttr  totalMaxAttr  baseArmor  minDmg  maxDmg  \
0        2.6    86.4             63         240.6       0.14      44      54   
1        2.8    95.2             63         221.4       2.43      53      61   
2        3.1    99.4             65         242.6       1.29      38      45   
3        1.7    58.8             65         242.6       3.43      57      63   
4        1.8    61.2             49         200.2      -0.29      45      55   

   range  baseAttackTime  attack

So, it only 16 features left which gonna use as input of the the neural network.

## Prepare the Data

At this stage, the `pandas dataFrame` will change to `NumPy` arrays. The data from column 2 to last will be use as input and column 1 as output/target which later it will be converted to one-hot vector: $STR\ \rightarrow \left[1, 0, 0 \right]$, $AGI\ \rightarrow \left[0, 1, 0 \right]$ and $INT\ \rightarrow \left[0, 0, 1\right]$ so it can be use as the output of the model.

In [7]:
# Get Pandas array value (Convert to NumPy array)
train_data = sf_train.values
val_data = sf_val.values

In [8]:
# Use columns 2 to last as Input
train_x = train_data[:,2:]
val_x = val_data[:,2:]

In [9]:
train_x

array([[23, 2.6, 85.4, ..., 150, 1.7, 0.56],
       [18, 1.7, 58.8, ..., 675, 1.7, 0.45],
       [22, 1.5, 58.0, ..., 150, 1.45, 0.3],
       ...,
       [16, 2.1, 66.4, ..., 600, 1.7, 0.4],
       [22, 2.8, 89.2, ..., 150, 1.7, 0.56],
       [19, 2.6, 81.4, ..., 350, 1.7, 0.633]], dtype=object)

In [10]:
train_x.shape

(99, 16)

In [11]:
val_x

array([[24, 3.0, 96.0, 1.8, 58.2, 24, 2.6, 86.4, 63, 240.6, 0.14, 44, 54,
        625, 1.7, 0.3],
       [25, 2.6, 87.4, 1.2, 38.8, 28, 2.8, 95.2, 63, 221.4, 2.43, 53, 61,
        400, 1.7, 0.4],
       [24, 2.4, 81.6, 1.9, 61.6, 25, 3.1, 99.4, 65, 242.6, 1.29, 38, 45,
        425, 1.7, 0.25],
       [23, 2.7, 87.8, 3.0, 96.0, 18, 1.7, 58.8, 65, 242.6, 3.43, 57, 63,
        150, 1.7, 0.43],
       [19, 3.2, 95.8, 1.3, 43.2, 18, 1.8, 61.2, 49, 200.2, -0.29, 45,
        55, 500, 1.7, 0.35],
       [17, 2.2, 69.8, 1.6, 52.4, 23, 1.7, 63.8, 54, 186.0, 0.0, 43, 52,
        575, 1.7, 0.15],
       [16, 1.8, 59.2, 1.7, 63.8, 26, 3.0, 98.0, 65, 221.0, 3.29, 41, 45,
        600, 1.7, 0.4],
       [19, 2.6, 81.4, 2.0, 72.0, 26, 2.7, 90.8, 69, 244.2, 3.93, 40, 55,
        450, 1.7, 0.46],
       [23, 3.2, 99.8, 1.95, 68.8, 14, 1.25, 44.0, 59, 212.6, 2.14, 52,
        59, 150, 1.7, 0.35],
       [23, 2.6, 85.4, 2.1, 73.4, 18, 1.7, 58.8, 64, 217.6, 3.29, 50, 54,
        150, 1.7, 0.36],
       [15,

In [12]:
val_x.shape

(16, 16)

In [13]:
# Use columns 1 as output/target (one-hot encoding)
train_y = to_categorical(train_data[:,1])
val_y = to_categorical(val_data[:,1])

## Create The Model

The model that created has **16 neurons in the input layer** to represented the character stats, **10 neurons in the hidden layer with Sigmoid activation** and **3 neurons** representing (STR, AGI and INT) in the **output layer with Softmax activation** and the output of the model is the ***probability distribution of the entire target value***.

In [14]:
# Create network
inputs = Input(shape=(16,))
h_layer = Dense(18, activation='sigmoid')(inputs)

# Softmax activation for multiclass classification
outputs = Dense(3, activation='softmax')(h_layer)

In [15]:
model = Model(inputs=inputs, outputs=outputs)

# Optimizer / update rule
sgd = SGD(lr=0.001)

# Compile the model with cross entropy loss
model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

Suppose our model is 100% sure if hero A is STR, then the output of the model is $\left[1, 0, 0 \right]$ and if our model is 50% sure if hero B is AGI, 25% sure if hero B is STR or INT, then the output from our model is $\left[0.25, 0.5, 0.25 \right]$.

***It must be remembered that the total value of the probability distribution for all targets is 1***

For the **Loss Function**, **Cross Entropy** will be used and the **Optimizer**, it will still used **SGD** with the amount of **learning rate is 0.001**. For the references, metrics accuracy was added to see how good the model for classification.

In [16]:
# Train the model and use validation data
model.fit(train_x, train_y, batch_size=32, epochs=10000, verbose=1, validation_data=(val_x, val_y))
model.save_weights('5_hero_classification_dota_2.h5')

Train on 99 samples, validate on 16 samples
Epoch 1/10000
99/99 [==============================] - 0s 1ms/sample - loss: 1.1067 - acc: 0.6061 - val_loss: 1.0539 - val_acc: 0.6250
Epoch 2/10000
99/99 [==============================] - 0s 163us/sample - loss: 1.0977 - acc: 0.5960 - val_loss: 1.0543 - val_acc: 0.6250
Epoch 3/10000
99/99 [==============================] - 0s 103us/sample - loss: 1.0907 - acc: 0.5960 - val_loss: 1.0548 - val_acc: 0.6250
Epoch 4/10000
99/99 [==============================] - 0s 132us/sample - loss: 1.0815 - acc: 0.5960 - val_loss: 1.0589 - val_acc: 0.6250
Epoch 5/10000
99/99 [==============================] - 0s 111us/sample - loss: 1.0731 - acc: 0.5960 - val_loss: 1.0595 - val_acc: 0.6250
Epoch 6/10000
99/99 [==============================] - 0s 114us/sample - loss: 1.0706 - acc: 0.5960 - val_loss: 1.0598 - val_acc: 0.6250
Epoch 7/10000
99/99 [==============================] - 0s 114us/sample - loss: 1.0697 - acc: 0.5960 - val_loss: 1.0575 - val_acc: 0.6250

99/99 [==============================] - 0s 112us/sample - loss: 0.9912 - acc: 0.5354 - val_loss: 1.0379 - val_acc: 0.5000
Epoch 61/10000
99/99 [==============================] - 0s 110us/sample - loss: 0.9881 - acc: 0.5455 - val_loss: 1.0463 - val_acc: 0.5000
Epoch 62/10000
99/99 [==============================] - 0s 137us/sample - loss: 0.9931 - acc: 0.5657 - val_loss: 1.0733 - val_acc: 0.5000
Epoch 63/10000
99/99 [==============================] - 0s 76us/sample - loss: 1.0331 - acc: 0.5556 - val_loss: 1.0661 - val_acc: 0.5000
Epoch 64/10000
99/99 [==============================] - 0s 115us/sample - loss: 1.0279 - acc: 0.5556 - val_loss: 1.0594 - val_acc: 0.5000
Epoch 65/10000
99/99 [==============================] - 0s 113us/sample - loss: 1.0212 - acc: 0.5354 - val_loss: 1.0482 - val_acc: 0.5000
Epoch 66/10000
99/99 [==============================] - 0s 96us/sample - loss: 1.0055 - acc: 0.5556 - val_loss: 1.0340 - val_acc: 0.5000
Epoch 67/10000
99/99 [=============================

99/99 [==============================] - 0s 93us/sample - loss: 0.9390 - acc: 0.7172 - val_loss: 1.0031 - val_acc: 0.5625
Epoch 120/10000
99/99 [==============================] - 0s 99us/sample - loss: 0.9401 - acc: 0.7273 - val_loss: 1.0178 - val_acc: 0.5625
Epoch 121/10000
99/99 [==============================] - 0s 109us/sample - loss: 0.9532 - acc: 0.6566 - val_loss: 1.0582 - val_acc: 0.3750
Epoch 122/10000
99/99 [==============================] - 0s 89us/sample - loss: 0.9820 - acc: 0.5657 - val_loss: 1.0545 - val_acc: 0.4375
Epoch 123/10000
99/99 [==============================] - 0s 106us/sample - loss: 0.9735 - acc: 0.5960 - val_loss: 1.0045 - val_acc: 0.5625
Epoch 124/10000
99/99 [==============================] - 0s 124us/sample - loss: 0.9383 - acc: 0.7071 - val_loss: 1.0073 - val_acc: 0.5625
Epoch 125/10000
99/99 [==============================] - 0s 103us/sample - loss: 0.9417 - acc: 0.7071 - val_loss: 1.0025 - val_acc: 0.5625
Epoch 126/10000
99/99 [=======================

99/99 [==============================] - 0s 127us/sample - loss: 0.9029 - acc: 0.7778 - val_loss: 1.0010 - val_acc: 0.5625
Epoch 179/10000
99/99 [==============================] - 0s 156us/sample - loss: 0.9039 - acc: 0.7576 - val_loss: 0.9978 - val_acc: 0.5625
Epoch 180/10000
99/99 [==============================] - 0s 111us/sample - loss: 0.9052 - acc: 0.7677 - val_loss: 0.9973 - val_acc: 0.6250
Epoch 181/10000
99/99 [==============================] - 0s 94us/sample - loss: 0.9044 - acc: 0.7576 - val_loss: 0.9931 - val_acc: 0.5625
Epoch 182/10000
99/99 [==============================] - 0s 107us/sample - loss: 0.9031 - acc: 0.7576 - val_loss: 0.9937 - val_acc: 0.5625
Epoch 183/10000
99/99 [==============================] - 0s 103us/sample - loss: 0.8998 - acc: 0.7980 - val_loss: 0.9963 - val_acc: 0.6250
Epoch 184/10000
99/99 [==============================] - 0s 112us/sample - loss: 0.9026 - acc: 0.7677 - val_loss: 0.9934 - val_acc: 0.6250
Epoch 185/10000
99/99 [=====================

99/99 [==============================] - 0s 117us/sample - loss: 0.8669 - acc: 0.8485 - val_loss: 0.9597 - val_acc: 0.6875
Epoch 238/10000
99/99 [==============================] - 0s 121us/sample - loss: 0.8669 - acc: 0.8384 - val_loss: 0.9578 - val_acc: 0.6250
Epoch 239/10000
99/99 [==============================] - 0s 89us/sample - loss: 0.8666 - acc: 0.8485 - val_loss: 0.9536 - val_acc: 0.6875
Epoch 240/10000
99/99 [==============================] - 0s 92us/sample - loss: 0.8703 - acc: 0.8384 - val_loss: 0.9517 - val_acc: 0.6875
Epoch 241/10000
99/99 [==============================] - 0s 96us/sample - loss: 0.8709 - acc: 0.8384 - val_loss: 0.9558 - val_acc: 0.6875
Epoch 242/10000
99/99 [==============================] - 0s 105us/sample - loss: 0.8646 - acc: 0.8384 - val_loss: 0.9696 - val_acc: 0.6250
Epoch 243/10000
99/99 [==============================] - 0s 121us/sample - loss: 0.8664 - acc: 0.8384 - val_loss: 0.9535 - val_acc: 0.6250
Epoch 244/10000
99/99 [=======================

...

99/99 [==============================] - 0s 99us/sample - loss: 0.2983 - acc: 0.9293 - val_loss: 0.8890 - val_acc: 0.7500
Epoch 9755/10000
99/99 [==============================] - 0s 144us/sample - loss: 0.2991 - acc: 0.9293 - val_loss: 0.8868 - val_acc: 0.7500
Epoch 9756/10000
99/99 [==============================] - 0s 126us/sample - loss: 0.2951 - acc: 0.9293 - val_loss: 0.8402 - val_acc: 0.7500
Epoch 9757/10000
99/99 [==============================] - 0s 129us/sample - loss: 0.3308 - acc: 0.8990 - val_loss: 0.8410 - val_acc: 0.7500
Epoch 9758/10000
99/99 [==============================] - 0s 114us/sample - loss: 0.3003 - acc: 0.9293 - val_loss: 0.8816 - val_acc: 0.7500
Epoch 9759/10000
99/99 [==============================] - 0s 118us/sample - loss: 0.2956 - acc: 0.9293 - val_loss: 0.8989 - val_acc: 0.7500
Epoch 9760/10000
99/99 [==============================] - 0s 150us/sample - loss: 0.3021 - acc: 0.9192 - val_loss: 0.8694 - val_acc: 0.7500
Epoch 9761/10000
99/99 [==============

Epoch 9813/10000
99/99 [==============================] - 0s 114us/sample - loss: 0.2944 - acc: 0.9293 - val_loss: 0.8800 - val_acc: 0.7500
Epoch 9814/10000
99/99 [==============================] - 0s 133us/sample - loss: 0.2984 - acc: 0.9293 - val_loss: 0.8820 - val_acc: 0.7500
Epoch 9815/10000
99/99 [==============================] - 0s 129us/sample - loss: 0.2979 - acc: 0.9293 - val_loss: 0.8882 - val_acc: 0.7500
Epoch 9816/10000
99/99 [==============================] - 0s 139us/sample - loss: 0.2945 - acc: 0.9293 - val_loss: 0.9033 - val_acc: 0.7500
Epoch 9817/10000
99/99 [==============================] - 0s 132us/sample - loss: 0.3006 - acc: 0.9192 - val_loss: 0.8862 - val_acc: 0.7500
Epoch 9818/10000
99/99 [==============================] - 0s 93us/sample - loss: 0.2965 - acc: 0.9293 - val_loss: 0.8904 - val_acc: 0.7500
Epoch 9819/10000
99/99 [==============================] - 0s 84us/sample - loss: 0.3090 - acc: 0.9192 - val_loss: 0.8521 - val_acc: 0.7500
Epoch 9820/10000
99/99

99/99 [==============================] - 0s 124us/sample - loss: 0.3004 - acc: 0.9192 - val_loss: 0.8901 - val_acc: 0.7500
Epoch 9872/10000
99/99 [==============================] - 0s 90us/sample - loss: 0.3004 - acc: 0.9192 - val_loss: 0.8806 - val_acc: 0.7500
Epoch 9873/10000
99/99 [==============================] - 0s 106us/sample - loss: 0.2981 - acc: 0.9192 - val_loss: 0.8798 - val_acc: 0.7500
Epoch 9874/10000
99/99 [==============================] - 0s 116us/sample - loss: 0.3025 - acc: 0.9192 - val_loss: 0.8843 - val_acc: 0.7500
Epoch 9875/10000
99/99 [==============================] - 0s 116us/sample - loss: 0.2945 - acc: 0.9293 - val_loss: 0.9153 - val_acc: 0.7500
Epoch 9876/10000
99/99 [==============================] - 0s 118us/sample - loss: 0.3134 - acc: 0.9192 - val_loss: 0.8855 - val_acc: 0.7500
Epoch 9877/10000
99/99 [==============================] - 0s 99us/sample - loss: 0.2946 - acc: 0.9293 - val_loss: 0.8856 - val_acc: 0.7500
Epoch 9878/10000
99/99 [===============

Epoch 9930/10000
99/99 [==============================] - 0s 118us/sample - loss: 0.2920 - acc: 0.9293 - val_loss: 0.9984 - val_acc: 0.6875
Epoch 9931/10000
99/99 [==============================] - 0s 84us/sample - loss: 0.3544 - acc: 0.8889 - val_loss: 0.8990 - val_acc: 0.7500
Epoch 9932/10000
99/99 [==============================] - 0s 119us/sample - loss: 0.2932 - acc: 0.9293 - val_loss: 0.8732 - val_acc: 0.7500
Epoch 9933/10000
99/99 [==============================] - 0s 134us/sample - loss: 0.3679 - acc: 0.8788 - val_loss: 0.8046 - val_acc: 0.8125
Epoch 9934/10000
99/99 [==============================] - 0s 140us/sample - loss: 0.3140 - acc: 0.9091 - val_loss: 0.8786 - val_acc: 0.7500
Epoch 9935/10000
99/99 [==============================] - 0s 139us/sample - loss: 0.2990 - acc: 0.9192 - val_loss: 0.8905 - val_acc: 0.7500
Epoch 9936/10000
99/99 [==============================] - 0s 118us/sample - loss: 0.2984 - acc: 0.9293 - val_loss: 0.8931 - val_acc: 0.7500
Epoch 9937/10000
99/9

99/99 [==============================] - 0s 121us/sample - loss: 0.3148 - acc: 0.9192 - val_loss: 0.8859 - val_acc: 0.7500
Epoch 9989/10000
99/99 [==============================] - 0s 114us/sample - loss: 0.3082 - acc: 0.9192 - val_loss: 0.8906 - val_acc: 0.7500
Epoch 9990/10000
99/99 [==============================] - 0s 118us/sample - loss: 0.3086 - acc: 0.9192 - val_loss: 0.8710 - val_acc: 0.7500
Epoch 9991/10000
99/99 [==============================] - 0s 110us/sample - loss: 0.3677 - acc: 0.8889 - val_loss: 0.8471 - val_acc: 0.7500
Epoch 9992/10000
99/99 [==============================] - 0s 100us/sample - loss: 0.3373 - acc: 0.8889 - val_loss: 0.8458 - val_acc: 0.7500
Epoch 9993/10000
99/99 [==============================] - 0s 109us/sample - loss: 0.3005 - acc: 0.9192 - val_loss: 0.9179 - val_acc: 0.7500
Epoch 9994/10000
99/99 [==============================] - 0s 157us/sample - loss: 0.3123 - acc: 0.8990 - val_loss: 0.8821 - val_acc: 0.7500
Epoch 9995/10000
99/99 [=============

The Next thing is training and evaluation must be done. When it finished, save the weight and bias into a file which it can be used again later. When it tried, The percentage of accuracy is 90% for training and accuracy for validation data is 75% after 5000 epochs.

In [17]:
# Predict all Validation data
predict = model.predict(val_x)

In [18]:
# Visualize Prediction
df = pd.DataFrame(predict)
df.columns = ['Strength', 'Agility', 'Intelligent']
df.index = val_data[:,0]
print(df)

                  Strength   Agility  Intelligent
ArcWarden         0.023553  0.063867     0.912580
Jakiro            0.023553  0.063867     0.912580
WinterWyvern      0.023553  0.063867     0.912580
Bloodseeker       0.091780  0.859415     0.048806
Phoenix           0.023554  0.063873     0.912574
Io                0.023748  0.065142     0.911110
Leshrac           0.023553  0.063867     0.912580
OutworldDevourer  0.023553  0.063868     0.912579
Brewmaster        0.804832  0.160901     0.034267
Tusk              0.138074  0.807997     0.053930
Weaver            0.030199  0.804785     0.165015
Undying           0.756342  0.015379     0.228279
Alchemist         0.744426  0.016625     0.238949
Phantom Lancer    0.084383  0.867885     0.047731
CrystalMaiden     0.023553  0.063867     0.912580
Invoker           0.023553  0.063867     0.912580


There was prediction errors in the results above, such as the ***Arc Warden which should be AGI Hero, Phoenix and IO should be STR Hero and so the Tuskar***.

There was times when the model cannot confidently guess the hero type, this can be seen in Weaver and Undying, where the difference in probability between the three types was not much different.